In [1]:
import os

In [2]:
%pwd

'/home/cdot/Documents/ML_Folder/folders/ML_for_MLFlow/research'

In [3]:
os.chdir("../")

In [24]:
%pwd

'/home/cdot/Documents/ML_Folder/folders/ML_for_MLFlow'

##### Update config

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

##### Configuration manager from src

In [26]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        return data_ingestion_config

##### Update Components

In [28]:
import os
import zipfile
import urllib.request as request
from ML_Project import logger
from ML_Project.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            size_file = get_size(Path(self.config.local_data_file))
            logger.info(f"file already exists of size: {size_file}")

    def extract_zip_file(self):
        """
        zip file path : str
        extract zip file into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

##### Update the Pipeline

In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
            

[2024-03-26 16:45:07,578: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-26 16:45:07,582: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-26 16:45:07,584: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-26 16:45:07,586: INFO: common: created directory at: artifacts]
[2024-03-26 16:45:07,587: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-26 16:45:07,961: INFO: 320765567: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E20A:16433C:22CF73:2F5DAF:6602AD9A
Accept-Ranges: bytes
Date: Tue, 26 